# Supply Chain Overstock Allocation Model

Here’s a quick dataset draft using simple warehouse data that can be used to create an algorithm to solve this supply chain allocation issue. (Please read my blog to learn more about this translated algorithm: https://itskathleenlara.medium.com/supply-chain-overstock-allocation-model-a37619eb4f4)

In [1]:
import pandas as pd

In [2]:
d = {'warehouse_id': [1, 1, 1, 2,2,2], 
     'store_id': ['str1', 'str2', 'str3','str1', 'str2', 'str3'],
     'store_bal':[50, 6, 8, 35,5,54], 
     'wk_avg_disp': [5, 4, 4, 3,4,7], 
     'warehouse_shipments': [18, 18, 18, 500,500,500]} 
df = pd.DataFrame(data=d)
df

,warehouse_id,store_id,store_bal,wk_avg_disp,warehouse_shipments
0,1,str1,50,5,18
1,1,str2,6,4,18
2,1,str3,8,4,18
3,2,str1,35,3,500
4,2,str2,5,4,500
5,2,str3,54,7,500


## Function Approach

In [3]:
df['wk_supply'] = 0
df['to_ship'] = 0

def algorithm(x):
    for i in range(x.iloc[0]['warehouse_shipments'])[0:500]:
          x['wk_supply'] = (x['store_bal']+1) / x['wk_avg_disp']
          x['warehouse_shipments'] = x['warehouse_shipments']-1
          x = x.sort_values(["wk_supply","wk_avg_disp"], ascending=[True,False]).reset_index(drop=True)
          x.iloc[0, 2] = x['store_bal'][0] + 1
          x.iloc[0, 6] = x['to_ship'][0] + 1
    return x

df_final = df.groupby('warehouse_id').apply(algorithm).reset_index(drop=True).sort_values(['warehouse_id', 'store_id'])
df_final

,warehouse_id,store_id,store_bal,wk_avg_disp,warehouse_shipments,wk_supply,to_ship
2,1,str1,50,5,0,10.200000,0
0,1,str2,16,4,0,4.000000,10
1,1,str3,16,4,0,4.250000,8
5,2,str1,127,3,0,42.666667,92
3,2,str2,170,4,0,42.500000,165
4,2,str3,297,7,0,42.571429,243


## While loop and Function Approach

In [4]:
df = pd.DataFrame({'warehouse_id': ['1', '1', '1', '2', '2', '2'],
          'store_id': ['str1', 'str2', 'str3', 'str1', 'str2', 'str3'],
          'store_bal': [50, 6, 8, 35, 5, 54],
          'wk_avg_disp': [5, 4, 4, 3, 4, 7],
          'warehouse_shipments': [18, 18, 18, 500, 500, 500]})
df

,warehouse_id,store_id,store_bal,wk_avg_disp,warehouse_shipments
0,1,str1,50,5,18
1,1,str2,6,4,18
2,1,str3,8,4,18
3,2,str1,35,3,500
4,2,str2,5,4,500
5,2,str3,54,7,500


In [5]:
def find_min(x):
  num_mins = x["wk_supply"].loc[x["wk_supply"] == x["wk_supply"].min()].shape[0]
  if num_mins == 1:
    return(x["wk_supply"].idxmin())
  else:
    min_df = x.loc[x["wk_supply"] == x["wk_supply"].min()]
  return(min_df["wk_avg_disp"].idxmax())

In [6]:
df['to_ship'] = 0
df['wk_supply'] = (df['store_bal']+1) / df['wk_avg_disp']
  
while (df['warehouse_shipments'] != 0).any():
  lowest_idx = df[df['warehouse_shipments'] > 0].groupby(["warehouse_id"])['wk_supply'].idxmin()
  df.loc[lowest_idx, 'to_ship'] += 1
  df['wk_supply'] = ((df['store_bal']+1) + df['to_ship']) / df['wk_avg_disp']
  df.loc[df['warehouse_shipments'] > 0, 'warehouse_shipments'] -= 1

In [7]:
df

,warehouse_id,store_id,store_bal,wk_avg_disp,warehouse_shipments,to_ship,wk_supply
0,1,str1,50,5,0,0,10.200000
1,1,str2,6,4,0,10,4.250000
2,1,str3,8,4,0,8,4.250000
3,2,str1,35,3,0,92,42.666667
4,2,str2,5,4,0,165,42.750000
5,2,str3,54,7,0,243,42.571429
